**Checklist**

- check for dupes in ingestion? 
    - not needed, no dupes, and text dupes should be checked after cleaning
- use textBlob as yet another way to perform basic sentiment analysis?
    - yes, TODO, but maybe not during cleanup
- create an incremental cleanup module?
    - overkill, just reprocess everything bc of dupes, 
      script takes 8 secs for 20k tweets, that's ~6 min for a million if linear
- improve script?
    - add logs?

In [1]:
import os
import re
import json
import time

import string
import datetime
import urlextract
import pandas as pd

from html import unescape
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [2]:
def load_todays_data():
    filepath = os.path.join("..","data","1_raw","tweets")
    today_prefix = datetime.datetime.now().strftime("%Y%m%d")
    dfm = []
    for f in os.listdir(filepath):
        if re.match(today_prefix, f):
            dfm.append(pd.read_csv(os.path.join(filepath, f)))
    df = pd.concat(dfm)
    df = df.reset_index(drop=True)
    return df

In [3]:
df = load_todays_data()

In [4]:
df.shape

(24800, 5)

In [5]:
# test for duplicated IDs
ids = df["ID"]
df[ids.isin(ids[ids.duplicated()])].shape

(0, 5)

In [6]:
# test for duplicated Text
txt = df["Text"]
df[txt.isin(txt[txt.duplicated()])].shape

(6358, 5)

In [7]:
# look at users with more than 1 tweet?
grouped = df[['User', 'ID']].groupby('User').count().sort_values('ID', ascending=False)

grouped[grouped['ID']>1].head()

ID
User               
AlluArjun373788   9
loveme134340x     6
CParambarai       4
ezeaccountant80   3
Lady_Minna        3

#### Adding Retweet Column

In [8]:
def is_retweet(col):

    for i in range(len(col)):
        if re.match(r'^RT', col) is not None:
            return 1
        else:
            return 0      
        
def map_is_retweet(col):
   
    bool_map = map(lambda x: is_retweet(x), col)       
    return(list(bool_map)) 

In [9]:
df['Retweet'] = map_is_retweet(df['Text'].values)

## Cleanup

In [10]:
def cleanup_tweet(tweet):
    """Cleans up a tweet with the following steps:
        1. make lower case
        2. remove URLs
        3. unescape HTML entities
        4. remove user references (including username) or hashtags, etc.
        5. remove punctuation
        6. remove emojis
        7. discard non-ascii decodable text after utf-8 encoding
        8. tokenize
        9. filter stop words from tokens
        10. stem filtered tokens
        
    The function returns a 3-tuple with cleaned versions 8 through 10.
    """
    # 1
    tweet = tweet.lower()

    # 2
    # URL_pattern = r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*'
    # tweet = re.sub(URL_pattern, '', tweet, flags=re.MULTILINE)
    # better, albeit slower, version
    urls = list(set(url_extractor.find_urls(tweet)))
    if len(urls) > 0:
        for url in urls:
            tweet = tweet.replace(url, "")
    # 3
    tweet = unescape(tweet)
    
    # 4
    pattern = r'\@\w+|\#|\¥|\â|\«|\»|\Ñ|\Ð|\¼|\½|\¾|\!|\?|\¿\
                |\x82|\x83|\x84|\x85|\x86|\x87|\x88|\x89|\
                |\x8a|\x8b|\x8c|\x8d|\x8e|\°|\µ|\´|\º|\¹|\³'
    tweet = re.sub(pattern, '', tweet)

    # 5
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    # 6
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet).strip()

    # 7
    def is_ascii(text):
        try:
            text.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        else:
            return True
    
    if is_ascii(tweet) == False:
        return " "
    else:
        pass

    # 8 tokenized only (remove retweet prefix)
    tweet_tokens = word_tokenize(tweet)
    retweet = ['rt']
    tweet_tokens = [token for token in tweet_tokens if not token in retweet]
    
    # 9 tokenized + filtered
    # NLTK's set(stopwords.words('english')) removes too many words
    # using list of 25 semantically non-selective words (Reuters-RCV1 dataset)
    stop_words = ['a','an','and','are','as','at','be','by','for','from',
                  'has','he','in','is','it','its','of','on','that','the',
                  'to','was','were','will','with'] 
    filtered_tokens = [token for token in tweet_tokens if not token in stop_words]

    # 10 tokenized + filtered + stemmed
    ps = PorterStemmer()
    filtered_stemmed_tokens = [ps.stem(token) for token in filtered_tokens]
        
    v8 = " ".join(tweet_tokens)
    v9 = " ".join(filtered_tokens)
    v10 = " ".join(filtered_stemmed_tokens)  
    
    return (v8, v9, v10)

In [11]:
url_extractor = urlextract.URLExtract()
tuples = [cleanup_tweet(tweet) for tweet in df.loc[:,'Text']]

In [12]:
df.loc[:, 'tokenized'], df.loc[:, 'filtered'], df.loc[:, 'stemmed'] = \
[x[0] for x in tuples], [x[1] for x in tuples], [x[2] for x in tuples]

In [13]:
df.head()

ID            Timestamp           User  \
0  1305170993015644160  2020-09-13 15:46:28    SassySnipez   
1  1305170992952565760  2020-09-13 15:46:28    nunubestboy   
2  1305170992352825344  2020-09-13 15:46:28    irisskhryss   
3  1305170992151621638  2020-09-13 15:46:28     shalexusss   
4  1305170991841116170  2020-09-13 15:46:28  codepinkanime   

                                                Text  Polarity  Retweet  \
0  Thinking about cutting my hair but idk 🤔 #NewP...        -1        0   
1  RT @milkypmh: thank you for working hard for t...        -1        1   
2  RT @chicheesticks: ya'll actually finished the...        -1        1   
3  RT @PrincessTaaaty: I need a date night 🥺, whe...        -1        1   
4  @RyTanaka2 @lucysupremacy @KawasPhattyCake @0I...        -1        0   

                                           tokenized  \
0  thinking about cutting my hair but idk newprof...   
1  thank you for working hard for this debut ever...   
2             yall actually finished the tommy video   
3     i need a date night where we talk eat and vibe   
4                                 i love you so much   

                                            filtered  \
0  thinking about cutting my hair but idk newprof...   
1  thank you working hard this debut everyone so ...   
2                 yall actually finished tommy video   
3           i need date night where we talk eat vibe   
4                                 i love you so much   

                                             stemmed  
0        think about cut my hair but idk newprofilep  
1  thank you work hard thi debut everyon so excit...  
2                     yall actual finish tommi video  
3           i need date night where we talk eat vibe  
4                                 i love you so much

In [14]:
df.tail()

ID            Timestamp             User  \
24795  1305332767639711744  2020-09-14 02:29:18   CherylRGilmer1   
24796  1305332767530430469  2020-09-14 02:29:18  KareyandKareful   
24797  1305332767526354945  2020-09-14 02:29:18    earth2delisha   
24798  1305332767459172352  2020-09-14 02:29:18    JoEy_ChAvEz21   
24799  1305332767455154177  2020-09-14 02:29:18         Vickaric   

                                                    Text  Polarity  Retweet  \
24795  RT @TAftermath2020: Hell yeah! Trump says he w...         1        1   
24796  RT @GreggsBooks: Follow everyone who likes or ...         1        1   
24797  the blacker the berry....😉 https://t.co/7aM09o...         1        0   
24798  RT @thecheckdown: Larry Fitzgerald really hand...         1        1   
24799  RT @nowthisnews: The Rock sent a personalized ...         1        1   

                                               tokenized  \
24795  hell yeah trump says he wants biden to take a ...   
24796         follow everyone who likes or retweets this   
24797                              the blacker the berry   
24798  larry fitzgerald really handdelivered the ball...   
24799  the rock sent a personalized birthday message ...   

                                                filtered  \
24795  hell yeah trump says wants biden take drug tes...   
24796         follow everyone who likes or retweets this   
24797                                      blacker berry   
24798  larry fitzgerald really handdelivered ball ref...   
24799  rock sent personalized birthday message this 1...   

                                                 stemmed  
24795  hell yeah trump say want biden take drug test ...  
24796             follow everyon who like or retweet thi  
24797                                      blacker berri  
24798  larri fitzgerald realli handdeliv ball ref vet...  
24799  rock sent person birthday messag thi 100yearol...

In [15]:
dupes = df[df['tokenized'].duplicated(keep='first')]

In [16]:
dupes.shape[0]/df.shape[0] # % dupes

0.24084677419354839

In [17]:
dupes[dupes['Retweet']==1].shape[0]/dupes.shape[0] # % of dupes that are retweets

0.8861543612924828